In [1]:
from __future__ import print_function, division

In [2]:
import requests

url = 'https://www.elliman.com/search/for-sale/search-1?sdid=1&sid=37687331&sk=1'

response = requests.get(url)

In [3]:
response #check that website is accessible

<Response [200]>

In [4]:
from bs4 import BeautifulSoup

In [42]:
#use selenium to login to site(needed to get all listings)
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import os
chromedriver = "/Applications/chromedriver" 
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)
driver.get("https://www.elliman.com/login")

In [43]:
username_form = driver.find_element_by_id("email_address_login")
username_form
username_form.send_keys("*****") #enter email

In [44]:
password_form=driver.find_element_by_id("password_login") 
password_form.send_keys('*****') # enter password

In [45]:
password_form.send_keys(Keys.RETURN)

In [10]:
#this function gets all links for a given search page
def get_links_from_page(current_url):
    driver.get(current_url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    all_listings = soup.find_all(class_="listing_address first")
    new_urls = []
    for item in all_listings:
        new_urls.append("https://www.elliman.com"+item.a.get('href'))
    return new_urls    

In [11]:
#this function goes through all search pages and gets links from each page
def get_all_links(search_url):
    response = requests.get(search_url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    all_links = []
    all_links.append(get_links_from_page(search_url))
    while soup.find(class_="iui_paginator_next").a != None:
        links = soup.find_all('a')
        for item in links:
            if item.get('rel')==['next']:
                next_url = "https://www.elliman.com"+item.get('href')
        all_links.append(get_links_from_page(next_url))
        response = requests.get(next_url)
        page= response.text
        soup = BeautifulSoup(page,"lxml") 
    return all_links

In [12]:
listings_links = get_all_links("https://www.elliman.com/search/for-sale?sid=37713533")

In [48]:
listings_links

[['https://www.elliman.com/new-york-city/432-park-avenue-ph95-manhattan-bamsgsz',
  'https://www.elliman.com/new-york-city/520-park-avenue-52-manhattan-ojcxbdj',
  'https://www.elliman.com/new-york-city/111-west---57-street-ph72-manhattan-cculbyo',
  'https://www.elliman.com/new-york-city/555-west-25-street-manhattan-exprvdg',
  'https://www.elliman.com/new-york-city/33-east-74-street-th-manhattan-kcsbgzf',
  'https://www.elliman.com/new-york-city/432-park-avenue-ph94a-manhattan-nbkjcyk',
  'https://www.elliman.com/new-york-city/565-broome-street-phnorth-manhattan-jefqfht',
  'https://www.elliman.com/new-york-city/432-park-avenue-87b-manhattan-clwshlo',
  'https://www.elliman.com/new-york-city/111-murray-street-ph2-manhattan-etnqdll',
  'https://www.elliman.com/new-york-city/118-east-76-street-manhattan-basrlqe',
  'https://www.elliman.com/new-york-city/33-east-74-street-ph1-manhattan-tsfvnwy',
  'https://www.elliman.com/new-york-city/845-united-nations-plaza-82acd-manhattan-ivvelyw',


In [51]:
#to check that all listings were retrieved
print("Total search pages retrieved: ",len(listings_links))
total = 0
for lists in listings_links:
    total+=len(lists)
print("Total listings links: ", total)

Total search pages retrieved:  288
Total listings links:  8635


In [19]:
import numpy as np

In [15]:
#this function gets all the features from a given page
def get_features(current_url):
    driver.get(current_url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    try:
        building_desc = str(soup.find_all("meta")[4]).split('"')[1]
    except:
        building_desc = ""
    
    try:
        new_soup = soup.find(class_="w_listitem_description")
    except:
        return
    
    try:
        address = new_soup.find(class_='first listing_address').text
    except:
        return
    
    try:
        price = soup.find(class_='listing_price').text.split('$')[1]
    except:
        price=0
    
    try:
        monthly_tax = soup.find(class_='listing_RETAX').text.split('$')[1]
    except:
        monthly_tax=np.nan
    
    try:
        monthly_main = soup.find(class_="listing_maintenanceCC").text.split('$')[1]
    except:
        monthly_main = np.nan
    
    try:
        neighb = soup.find(class_='listing_neighborhood').next.next.text
    except:
        neighb = ""
        
    try:
        beds_baths = soup.find(class_='listing_features').text.strip('\n')
    except:
        beds_baths = 0
    
    try:
        extras = soup.find(class_="listing_extras").text.strip('\n')
    except:
        extras = ""
    
    try:
        sq_ft = new_soup.find(class_='listing_features').findNextSibling().text.split(':')[1]
    except:
        sq_ft=np.nan
    
    try:
        ext_sq_ft = new_soup.find(class_="listing_features").findNextSibling().findNextSibling().text.split(':')[1]
    except:
        ext_sq_ft = np.nan
    
    try:
        building_dets = get_building_details(new_soup)
    except:
        building_dets = ""
    
    headers = ["building_desc",'address','price','monthly_tax','monthly_main','neighb','beds_baths','extras','sq_ft','ext_sq_ft',"building_dets"]
    attributes = [building_desc,address,price,monthly_tax,monthly_main,neighb,beds_baths,extras,sq_ft,ext_sq_ft,building_dets]
    sales_data = []
    sales_dict = dict(zip(headers,attributes))
    return sales_dict

In [16]:
#this function gets the building details section from a given page
def get_building_details(my_soup):
    building_features = []
    x = my_soup.find_all('ul')[1]
    y = x.find_all('li')
    j=0
    if y[j]["class"] == ["", "first", "last"]:
        building_features.append(y[j].text)
    else:
        while y[j]["class"] != ["","last"]:
            building_features.append(y[j].text)
            j+=1
        building_features.append(y[j].text)
    return building_features

In [205]:
my_features = []
for page in listings_links[0:9]:
    for link in page:
        my_features.append(get_features(link))

In [20]:
import pandas as pd

In [17]:
def get_all_features(a,b):
    my_features = []
    for page in listings_links[a:b]:
        for link in page:
            if get_features(link)==None:
                continue
            else:
                my_features.append(get_features(link))
    my_df = pd.DataFrame(my_features)
    my_df.to_csv(str("listings_data"+str(a)+"-"+str(b)+".csv"))

In [255]:
get_all_features(9,10)

In [256]:
get_all_features(10,20)

In [257]:
get_all_features(20,30)

In [260]:
get_all_features(30,40)

In [264]:
get_all_features(40,50)

In [272]:
get_all_features(50,60)

In [273]:
get_all_features(60,70)

In [274]:
get_all_features(70,80)

In [277]:
get_all_features(80,90)

In [278]:
get_all_features(90,100)

In [279]:
get_all_features(100,110)

In [280]:
get_all_features(110,120)

In [281]:
get_all_features(120,130)

In [21]:
get_all_features(130,140)

In [22]:
get_all_features(140,150)

In [23]:
get_all_features(150,160)

In [24]:
get_all_features(160,170)

In [25]:
get_all_features(170,180)

In [26]:
get_all_features(180,190)

In [37]:
get_all_features(190,200)

In [38]:
get_all_features(200,210)

In [39]:
get_all_features(210,220)

In [40]:
get_all_features(220,230)

In [41]:
get_all_features(230,240)

In [46]:
get_all_features(240,250)

In [47]:
get_all_features(250,260)

In [48]:
get_all_features(260,270)

In [49]:
get_all_features(270,280)

In [50]:
get_all_features(280,289)